# Comparison of computational costs
## Distances and distance vectors
- Pytorch 
- Numpy
- Numba

In [ ]:
import numpy as np
from numba import jit
import torch

import timeit

In [ ]:
def dist_vec(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    dist = np.linalg.norm(
        x[:, None, :] - x[None, :, :],
        axis=-1)
    vecs = x[None, :, :] - x[:, None, :]       
    return dist, vecs

In [ ]:
@jit
def dist_jit(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    N, dim = x.shape[0], x.shape[-1]
    dist = np.zeros((N, N))
    vecs = np.zeros((N, N, dim))
    for i in range(x.shape[0]):
        for j in range(i):
            dist[i][j] = np.linalg.norm(x[i] - x[j])
            vecs[i][j] = x[i] - x[j]
            dist[j][i] = dist[i][j]
            vecs[j][i] = vecs[i][j]
    return dist, vecs

In [ ]:
def dist_torch(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x)
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1), vecs

In [ ]:
def dist_cuda(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x).cuda()
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1).cpu(), vecs.cpu()

In [ ]:
x = np.array([[0., 0.],
                [0., 1.],
                [1., 1.],
                [1., 0.]])
q = np.array([1., -1., 1., -1.])

In [ ]:
assert (dist_vec(x)[0] == dist_jit(x)[0]).any
assert (dist_vec(x)[1] == dist_jit(x)[1]).any
assert (dist_vec(x)[0] == dist_torch(x)[0].numpy()).any
assert (dist_vec(x)[1] == dist_torch(x)[1].numpy()).any
#assert (dist_vec(x)[0] == dist_cuda(x)[0].numpy()).any
#assert (dist_vec(x)[1] == dist_cuda(x)[1].numpy()).any

In [ ]:
import random

def Random_particles(N):
    """Creates a list of N particles with random positions and charges
    
    Arguments:
        N (int): number of particles
        
    Output:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
    """
    return np.random.rand(N,3), np.array([[-1,1][random.randrange(2)] for i in range(N)])


In [ ]:
N = 500
x, q = Random_particles(N)

In [ ]:
%timeit dist_torch(x)
#%timeit dist_cuda(x)
%timeit dist_vec(x)
%timeit dist_jit(x)

## Distances and distance vectors
- Numpy
- Numba

In [ ]:
def distances(x): # pytorch
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x)
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1), vecs

In [ ]:
@jit
def Coulomb_force_jit(x, q):
    """Coulomb's law

    Arguments:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
        
    Constants:
        vacuum permittivity: eps0 = 8.854187e-12 
        elementary charge: qe = 1.602177e-19
    
    Output:
        f (float): forces between all particle pairs (dim = N x N x 3)
    """
    eps0, qe = 1., 1.
    force = np.zeros(x.shape)
    dist, vecs = distances(x)[0].numpy(), distances(x)[1].numpy()
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            if dist[i][j] != 0:
                force[i] += q[i] * q[j] * vecs[i][j] / dist[i][j]**3
    return qe/(4*np.pi*eps0)*force

In [ ]:
def Coulomb_force_vec(x, q):
    """Coulomb's law

    Arguments:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
        
    Constants:
        vacuum permittivity: eps0 = 8.854187e-12 
        elementary charge: qe = 1.602177e-19
    
    Output:
        f (float): forces between all particle pairs (dim = N x N x 3)
    """
    eps0, qe = 1., 1.
    force = np.zeros(x.shape)
    dist, vecs = distances(x)
    dist[dist!=0] = 1/dist[dist!=0]**3
    force = np.dot(np.diag(q), vecs * dist[:, :, None])
    force = np.einsum("ijk,j", force, q)
    return qe/(4*np.pi*eps0)*force

In [ ]:
def coulomb(coord, q, eps0=1, pbc=False):
        dist, vectors = distances(coord)
        dist[dist!=0] = 1/dist[dist!=0]**3
        D = dist[:,:,None]*vectors
        return q[:, None]*np.einsum("ijk, j",D, q)

In [ ]:
positions = np.array([[0., 0.],
                    [0., 1.],
                    [1., 1.],
                    [1., 0.]])
charges = np.array([1., -1., 1., -1.])

In [ ]:
print(Coulomb_force_vec(positions, charges))
print(-Coulomb_force_jit(positions, charges))

In [ ]:
N = 100
x, q = Random_particles(N)

In [ ]:
%timeit Coulomb_force_vec(x, q)
%timeit Coulomb_force_jit(x, q)
%timeit coulomb(x, q)

## Lennard-Jones-Gradient
- Numpy 
- Pytorch 

In [ ]:
def dist_torch(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x)
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1), vecs

def gradLJ_t(x, sig=1, eps=1):
    dist, vecs = dist_torch(x)
    dist[dist!=0] = 1/dist[dist!=0]
    D_att = 6 * sig**6 * dist**8
    D_rep = -12 * sig**12 * dist**14
    D = 4*(eps*(D_att + D_rep))[:, :, None]*vecs
    return torch.sum(D, dim=-2)

# Felix
def vectors(coord, boxsize, pbc=False):
    vecs = coord[:, None, :] - coord[None, :, :]
    if not pbc:
        return vecs
    elif pbc:
        L = boxsize[1] - boxsize[0] #calculate boxlength
        vecs += (vecs<-0.5*L)*L - (vecs>0.5*L)*L
        return vecs

def distances(vectors):
    return np.linalg.norm(vectors,axis=-1)
    
def gradLJ(vecs, sig=1, eps=1):
    dist = distances(vecs)
    dist[dist!=0] = 1/dist[dist!=0]
    D_att = 6 * sig**6 * dist**8
    D_rep = -12 * sig**12 * dist**14
    D = 4*(eps*(D_att + D_rep))[:, :, None]*vecs
    return np.sum(D, axis=-2)

In [ ]:
x_init = Random_particles(100)[0]

In [ ]:
%timeit gradLJ_t(x_init).numpy()
%timeit gradLJ(vectors(x_init, boxsize=(0,1)))